<a href="https://colab.research.google.com/github/max-moore/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS18_213_Ridge_Regression_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [ ] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [ ] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [ ] Do one-hot encoding of categorical features.
- [ ] Do feature selection with `SelectKBest`.
- [ ] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [ ] Get mean absolute error for the test set.
- [ ] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [ ]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv',
                 parse_dates=['SALE DATE'],
                 index_col='SALE DATE')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

# Subsetting Data

In [ ]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [ ]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

In [ ]:
# Subsetting our data

df = df[df['BUILDING_CLASS_CATEGORY'] == '01 ONE FAMILY DWELLINGS']
df = df[df['SALE_PRICE'] > 100000]
df = df[df['SALE_PRICE'] < 2000000]

In [ ]:
# Take a quick look to make sure everything looks good

df

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE
SALE DATE,,,,,,,,,,,,,,,,,,,,
2019-01-01,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000
2019-01-01,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000
2019-01-02,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000
2019-01-02,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-04-30,4,OTHER,01 ONE FAMILY DWELLINGS,1,13215,3,NaN,A2,244-15 135 AVENUE,NaN,11422.0,1.0,0.0,1.0,"3,300",1478.0,1925.0,1,A2,635000
2019-04-30,4,OTHER,01 ONE FAMILY DWELLINGS,1,11612,73,NaN,A1,10919 132ND STREET,NaN,11420.0,1.0,0.0,1.0,"2,400",1280.0,1930.0,1,A1,514000
2019-04-30,4,OTHER,01 ONE FAMILY DWELLINGS,1,11808,50,NaN,A0,135-24 122ND STREET,NaN,11420.0,1.0,0.0,1.0,"4,000",1333.0,1945.0,1,A0,635000


# Train/Test Split

In [ ]:
# Changing land square feet from string to float

df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].replace(',','', regex=True).astype(float)

In [ ]:
# Creating train/test split

y = df['SALE_PRICE']

X = df.drop([ 'NEIGHBORHOOD',
             'BUILDING_CLASS_CATEGORY',
             'EASE-MENT',
             'ADDRESS',
             'APARTMENT_NUMBER',
             'SALE_PRICE'], 
             axis=1)

# Use data from Jan/Feb/March to train
# Use data from April to test

cutoff = '04/01/2019'
mask = X.index < cutoff

# Train split

X_train, y_train = X.loc[mask], y.loc[mask]

# Test split

X_val, y_val = X.loc[~mask], y.loc[~mask]

In [ ]:
# Checking shape, looks good

# Sanity check
assert X_train.shape[0] + X_val.shape[0] == X.shape[0]

In [ ]:
X_train

,BOROUGH,TAX_CLASS_AT_PRESENT,BLOCK,LOT,BUILDING_CLASS_AT_PRESENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE
SALE DATE,,,,,,,,,,,,,,
2019-01-01,3,1,5495,801,A9,11230.0,1.0,0.0,1.0,6800.0,1325.0,1930.0,1,A9
2019-01-01,4,1,7918,72,A1,11427.0,1.0,0.0,1.0,4000.0,2001.0,1940.0,1,A1
2019-01-02,2,1,4210,19,A1,10461.0,1.0,0.0,1.0,3500.0,2043.0,1925.0,1,A1
2019-01-02,3,1,5212,69,A1,11226.0,1.0,0.0,1.0,4000.0,2680.0,1899.0,1,A1
2019-01-02,3,1,7930,121,A5,11203.0,1.0,0.0,1.0,1710.0,1872.0,1940.0,1,A5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-03-29,5,1,4081,44,A2,10306.0,1.0,0.0,1.0,2400.0,921.0,1950.0,1,A2
2019-03-29,5,1,2373,201,A5,10314.0,1.0,0.0,1.0,2450.0,2128.0,1980.0,1,A5
2019-03-29,5,1,1132,42,A1,10302.0,1.0,0.0,1.0,4361.0,1807.0,2018.0,1,A1


# One Hot Encoding

In [ ]:
# Imports for one hot encoding

from category_encoders import OneHotEncoder
from sklearn.linear_model import LinearRegression

# SelectKBest

In [ ]:
from sklearn.feature_selection import SelectKBest

k_values = list(range(1,len(X_train.columns)+1))
r2_scores = []

for k in k_values:
  # Instantiate Transformer
  ohe = OneHotEncoder(cols=['BOROUGH',
                          'TAX_CLASS_AT_PRESENT',
                          'BUILDING_CLASS_AT_PRESENT',
                          'RESIDENTIAL_UNITS',
                          'COMMERCIAL_UNITS',
                          'TOTAL_UNITS',
                          'BUILDING_CLASS_AT_TIME_OF_SALE'], 
                    use_cat_names=True)

  # Transform Feature Matrix
  XT_train = ohe.fit_transform(X_train)

  # Instantiate Predictor
  model = LinearRegression()

  # Transform test set
  XT_val = ohe.transform(X_val)

  # Instantiate Transformer 2
  skb = SelectKBest(k=k)

  # Fit_transform 2
  XTT_train = skb.fit_transform(XT_train, y_train)
  XTT_val = skb.transform(XT_val)

  # Fit on the DOUBLE-TRANSFORMED data
  model.fit(XTT_train, y_train)

  # Get R^2 score
  r2_scores.append(model.score(XTT_train, y_train))

In [ ]:
# Note that we DON'T retrain the transformer
y_pred_val = model.predict(XTT_val)

# Ridge Regression

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
model2 = Ridge(normalize=True)
model2.fit(XT_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None, normalize=True,
      random_state=None, solver='auto', tol=0.001)

In [ ]:
model2.score(XT_val, y_val)

0.28731338451254773

# MAE

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_val, y_pred_val)

182539.18077072292